# Exploring Dataset


In [1]:
import pandas as pd
from datasets import load_dataset

train_dataset = load_dataset("tyqiangz/multilingual-sentiments", 'all', split='train')

0000.parquet:   0%|          | 0.00/37.1M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/270399 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10857 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14465 [00:00<?, ? examples/s]

In [ ]:
train_set = train_dataset.train_test_split()

In [ ]:
len(train_set)

2

In [2]:
df = pd.DataFrame(train_dataset.to_dict())
df

,text,source,language,label
0,yang memerlukan pemerhatian dan tindakan serius,malaya,malay,0
1,sentiasa memikirkan dan merancang inisiatif ba...,malaya,malay,0
2,Kita akan tengok daripada pelbagai aspek supay...,malaya,malay,0
3,justeru asean perlu mengambil tindakan sebagai...,malaya,malay,0
4,"@_Niiar_ Jangan punah dulu, aku belum ke labua...",malaya,malay,1
...,...,...,...,...
270394,RT @user: #أيفون_البروفيسورمسابقة على أيفون7 أ...,sem_eval_2017,arabic,1
270395,اللهم أنت السلام ومنك السلام تباركت يا ذا الجل...,sem_eval_2017,arabic,0
270396,على وقع حمى الانتخابات الأميركية ويكليكس تكشف ...,sem_eval_2017,arabic,2
270397,"@user @user على البي سي""ويندوز 10""",sem_eval_2017,arabic,1


In [10]:
from model import load_pretrained_gpt2_tokenizer

In [11]:
tokenizer = load_pretrained_gpt2_tokenizer()

In [13]:
tokenizer(df['text'][2500])

{'input_ids': [51, 316, 15042, 275, 10102, 393, 648, 300, 391, 1066, 263, 600, 993, 479, 5350, 479, 1045, 256, 461, 731, 377, 11, 256, 461, 1888, 283, 11, 256, 461, 16486, 84, 11, 256, 461, 1458, 315], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
df['text'][270398]

'عندي وحده تشبه هاري بوتر يا انها كيوتيه😨!😭💗'

In [ ]:
from 

# Test

In [1]:
from utils import (
    read_yaml,
    get_tokenizer_config,
    get_dataset_config,
    DEVICE,
)
from gpt_dataset import GPTDataset
from model import load_model, load_tokenizer, generate

In [2]:
config = read_yaml('./train_config.yaml')

tokenizer_name, tokenizer_path, tokenizer_config = get_tokenizer_config(config)
tokenizer = load_tokenizer(tokenizer_name, tokenizer_path)

dataset_path, dataset_desc, dataset_config = get_dataset_config(config)
train_dataset = GPTDataset(
    filepath=dataset_path, tokenizer=tokenizer, **dataset_config
)

Token indices sequence length is longer than the specified maximum sequence length for this model (5145 > 1024). Running this sequence through the model will result in indexing errors


In [3]:
with open('./dataset/the-verdict.txt', 'r') as F:
    file_text = F.read()
file_text[:10]

'I HAD alwa'

# Predict

In [16]:
import os
import random
from argparse import ArgumentParser
import logging

import torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

from utils import (
    read_yaml,
    get_model_config,
    get_tokenizer_config,
    get_split_config,
    get_dataset_config,
    get_trainer_config,
    get_prediction_config,
    get_lora_config,
    _handle_seed,
    DEVICE,
)
from sentiment_dataset import SentimentDataset
from model import (
    load_model,
    load_tokenizer,
    load_lora_model,
    predict,
    compute_metrics,
    METRICS_DICT,
)


In [17]:
config = read_yaml("./configs/train_config_llama.yaml")

In [18]:
tokenizer_name, tokenizer_path, tokenizer_config = get_tokenizer_config(config)
tokenizer = load_tokenizer(
    tokenizer_name=tokenizer_name,
    tokenizer_path=tokenizer_path,
    tokenizer_config=tokenizer_config,
)

In [4]:
dataset_desc, (train_split_config, val_split_config, test_split_config) = (
    get_split_config(config)
)

test_dataset_path, test_dataset_language, test_dataset_config = (
    get_dataset_config(test_split_config)
)
test_dataset = SentimentDataset(
    dataset_path=test_dataset_path,
    dataset_language=test_dataset_language,
    split_type="test",
    tokenizer=tokenizer,
    **test_dataset_config,
)

In [5]:
model_name, model_path, base_model_path, model_config = get_model_config(config)
model_config.update(dict(pad_token_id=tokenizer.pad_token_id))
model = load_model(
    model_string=model_name,
    model_path=model_path,
    base_model_path=base_model_path,
    model_config=model_config,
)
model = model.to(device=DEVICE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
prompt_samples = [
    "this is a bad product",
    "this the nice product",
    "this is a best product",
    "how are you?"
]

In [15]:
predcitions = predict(
    model, tokenizer, prompt_samples, device=DEVICE,
)
predcitions

(array([2, 0, 0, 1]), ['negative', 'positive', 'positive', 'neutral'])

In [13]:
import evaluate

In [9]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=test_dataset_config.get("padding", True),
    return_tensors="pt",
)

trainer = Trainer(
    model,
    data_collator=data_collator,
    compute_metrics=lambda x: (compute_metrics(x, {'accuracy': evaluate.load("accuracy")})),
)

In [10]:
trainer.evaluate(test_dataset)

  0%|          | 0/109 [00:00<?, ?it/s]

{'eval_loss': 2.099609375,
 'eval_model_preparation_time': 0.0041,
 'eval_accuracy': 0.73103,
 'eval_runtime': 46.9713,
 'eval_samples_per_second': 18.522,
 'eval_steps_per_second': 2.321}